In [1]:
import os
import json
import gradio as gr

import hana_ml
from hana_ml import dataframe
from hana_ml.dataframe import ConnectionContext
from hana_ml.algorithms.pal.utility import DataSets, Settings
import pandas as pd
from gen_ai_hub.proxy.native.openai import embeddings
from langchain import PromptTemplate
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')

c:\Users\cesco\anaconda3\envs\btpenv3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
with open(os.path.join(os.getcwd(),
'C:\workspace\hands-on\HANA-configure-jinwook.json')) as f:
    hana_env_c = json.load(f)
    port_c = hana_env_c['port']
    user_c = hana_env_c['user']
    host_c = hana_env_c['host']
    pwd_c = hana_env_c['pwd']

cc = ConnectionContext(address=host_c, port=port_c, user= user_c, password= pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

#Check Hana_ml version
print(cc.hana_version())
#Check current schema
print(cc.get_current_schema())

4.00.000.00.1713874676 (CE2024.14)
GEN_AI


In [7]:
# Get Embedding
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="dc872f9eef04c31a") -> str:
    response = embeddings.create(
        deployment_id=model,
        input=input
    )
    return response.data[0].embedding

In [35]:
# Wrapping HANA vector search in a function_kr
def run_vector_search_kr(query: str, metric="COSINE_SIMILARITY", k=5):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
        col = 'L2D_SIM'
    else:
        sort = 'DESC'
        col = 'COS_SIM'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "BlockID","MaterialNumber",
    "MaterialName","MaterialDescription_KR","PurchasingGroupDescription_KR",
    "Blocked_stock","Block_reason_KR","Blocked_date",
    "Solution_KR", "Solution_date",
    "{metric}"("VECTOR_EN", TO_REAL_VECTOR('{qv}')) AS "{col}"
    FROM "DEMO_BLOCK_JINWOOK"
    ORDER BY "{col}" {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort, col = col)
    hdf = cc.sql(sql)
    df_context = hdf.collect()
    return df_context

# Wrapping HANA vector search in a function_en
def run_vector_search_en(query: str, metric="COSINE_SIMILARITY", k=5):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
        col = 'L2D_SIM'
    else:
        sort = 'DESC'
        col = 'COS_SIM'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "BlockID","MaterialNumber",
    "MaterialName","MaterialDescription","PurchasingGroupDescription",
    "Blocked_stock","Block_reason","Blocked_date", "Solution",
    "Solution_date","{metric}"("VECTOR_EN", TO_REAL_VECTOR('{qv}')) AS "{col}"
    FROM "DEMO_BLOCK_JINWOOK"
    ORDER BY "{col}" {sort}'''.format(k=k, metric = metric, qv=query_vector, sort=sort, col = col)
    hdf = cc.sql(sql)
    df_context = hdf.collect()
    return df_context

In [36]:
# Test the vector search_en
query = "What is the quality issue for packing defects for cabinet?"
df_context_en = run_vector_search_en(query=query,metric="COSINE_SIMILARITY",k=5)
df_context_en

,BlockID,MaterialNumber,MaterialName,MaterialDescription,PurchasingGroupDescription,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_date,COS_SIM
0,2,250249,VITTSJ√ñ,"Laptop stand, 35x65 cm",Tables & desks,34657,Identification of packaging defects that compr...,2024-03-10,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-04-23,0.808054
1,10,69276256,M√ñCKELBY / HENRIKSDAL,"Table and 6 chairs, 235x100 cm",Tables & desks,3051,Identification of packaging defects that compr...,2024-06-06,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-09-03,0.798295
2,4,39213915,KLIMPEN,"Table, 150x75 cm",Tables & desks,21111,Critical feedback on packaging quality leading...,2024-06-20,"[{""Instruction_step"": 1, ""Instruction"": ""Gathe...",2024-06-27,0.796200
3,5,50391421,NORDLI,"Top and plinth, 40x47 cm",Chests of drawers & drawer units,23208,Discrepancies in item quantity received by cus...,2024-05-06,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-05-08,0.778709
4,7,29203889,PLATSA,"Wardrobe, 140x57x261 cm",Wardrobes,14629,Experiencing supplier delays that result in in...,2024-03-19,"[{""Instruction_step"": 1, ""Instruction"": ""Ident...",2024-07-07,0.772991


In [37]:
# Test the vector search_kr
query = "의자의 포장불량에 대한 문제는 무엇입니까?"
df_context_kr = run_vector_search_kr(query=query, metric="L2DISTANCE",k=5)
df_context_kr

,BlockID,MaterialNumber,MaterialName,MaterialDescription_KR,PurchasingGroupDescription_KR,Blocked_stock,Block_reason_KR,Blocked_date,Solution_KR,Solution_date,L2D_SIM
0,10,69276256,M√ñCKELBY / HENRIKSDAL,"테이블과 6개의 의자, 235x100 cm",테이블 & 책상,3051,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,2024-06-06,"[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-09-03,0.749722
1,2,250249,VITTSJ√ñ,"노트북 스탠드, 35x65 cm",테이블 & 책상,34657,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,2024-03-10,"[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-23,0.766198
2,6,80369112,NORDVIKEN,"등받이가 있는 바 의자, 75 cm",의자들,1200,고객이 받은 품목 수량의 불일치로 인해 불만과 주문 정정 요청이 발생합니다.,2024-05-23,"[{""Instruction_step"": 1, ""Instruction_KR"": ""주문...",2024-07-13,0.769580
3,4,39213915,KLIMPEN,"테이블, 150x75 cm",테이블 & 책상,21111,포장 품질에 대한 중요한 피드백이 포장 재료와 과정의 검토로 이어집니다.,2024-06-20,"[{""Instruction_step"": 1, ""Instruction_KR"": ""중요...",2024-06-27,0.774646
4,1,59829111,PO√ÑNG,발판,소파 & 안락의자,2695,안전 문제로 특정 제품 배치에 대한 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.,2024-01-20,"[{""Instruction_step"": 1, ""Instruction_KR"": ""영향...",2024-01-31,0.784527


In [38]:
promptTemplate_fstring = """
You are provided multiple context items that are related to the prompt
you have to answer.
Use the following pieces of context to answer the question at the end.
Context:
{context}

Question:
{query}

Answer:
To answer questions, Please provides solutions with detail instructions
with steps and ResponsibleDepartment.
You must format your output as a plain text that adheres in english.
This is example format:
1. Solution:
- Responsible department:
- Instructions:
1) , 2) , 3), ...
2. Solution:
- Responsible department:
- Instructions:
1) , 2) , 3), ...
...
"""
promptTemplate_fstring_KR = """
You are provided multiple context items that are related to the prompt
you have to answer.
Use the following pieces of context to answer the question at the end.
Context:
{context}

Question:
{query}

Answer:
To answer questions, Please provides Solution_KR with detail instructions
with steps and ResponsibleDepartment_KR for each step.
You must format your output as a json adheres in korean.
This is example format:
1. 해결방안:
- 담당부서:
- 해결절차:
1) , 2) , 3), ...
2. 해결방안:
- 담당부서:
- 해결절차:
1) , 2) , 3), ...
...
"""

In [39]:
from langchain import PromptTemplate
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')
promptTemplate = PromptTemplate.from_template(promptTemplate_fstring)
promptTemplate2 = PromptTemplate.from_template(promptTemplate_fstring_KR)

def ask_llm_en(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 5) -> str:
    class color:
        RED = '\033[91m'
        BLUE = '\033[94m'
        BOLD = '\033[1m'
        END = '\033[0m'
    context = ''
    if retrieval_augmented_generation == True:
        df_context = run_vector_search_en(query, metric, k)
        # print(color.RED + '\nHANA vector search returned {k} best matching documents.'.format(k=k) + color.END)
        # print(color.RED + '\nGenerating LLM prompt using the context information.' + color.END)
        print(df_context)
    else:
        print(color.RED + 'Generating LLM prompt WITHOUT context information.' + color.END)
        
    prompt = promptTemplate.format(query=query, context=''.join(df_context[["BlockID","MaterialNumber",
            "MaterialName","MaterialDescription","PurchasingGroupDescription",
            "Block_reason","Blocked_date", "Solution",
            "Solution_date"]].astype('string')))
    # print(color.RED + '\nAsking LLM...' + color.END)
    llm = ChatOpenAI(deployment_id="d03974e89ef130ad", temperature=0.2)
    response = llm.predict(prompt)
    
    print("PROPOSAL SOLUTIONS")
    print(response)

def ask_llm_kr(query: str, retrieval_augmented_generation: bool, metric='L2DISTANCE', k = 5) -> str:
    class color:
        RED = '\033[91m'
        BLUE = '\033[94m'
        BOLD = '\033[1m'
        END = '\033[0m'
    context = ''
    if retrieval_augmented_generation == True:
        df_context = run_vector_search_kr(query, metric, k)
        # print(color.RED + '\nHANA vector search returned {k} best matching documents.'.format(k=k) + color.END)
        # print(color.RED + '\nGenerating LLM prompt using the context information.' + color.END)
        print(df_context)
    else:
        print(color.RED + 'Generating LLM prompt WITHOUT context information.' + color.END)

    prompt = promptTemplate2.format(query=query, context=''.join(df_context[["BlockID","MaterialNumber",
            "MaterialName","MaterialDescription_KR","PurchasingGroupDescription_KR",
            "Block_reason_KR","Blocked_date", "Solution_KR",
            "Solution_date"]].astype('string')))
    # print(color.RED + '\nAsking LLM...' + color.END)
    llm = ChatOpenAI(deployment_id="d03974e89ef130ad", temperature=0.2)
    response = llm.predict(prompt)
    
    print("PROPOSAL SOLUTIONS")
    print(response)



In [40]:
# query = "가구 포장 불량 배송에 대한 해결방안은 무엇인가요?"
query = "가구 포장 불량 배송에 대한 문제는 무엇인가요?"
response = ask_llm_kr(query=query, retrieval_augmented_generation=True, k=5)
print(response)

   BlockID  MaterialNumber            MaterialName   MaterialDescription_KR  \
0       10        69276256  M√ñCKELBY / HENRIKSDAL  테이블과 6개의 의자, 235x100 cm   
1        7        29203889                  PLATSA        옷장, 140x57x261 cm   
2        3        10364523               KALLVIKEN              문, 60x64 cm   
3        6        80369112               NORDVIKEN      등받이가 있는 바 의자, 75 cm   
4        2          250249                VITTSJ√ñ        노트북 스탠드, 35x65 cm   

  PurchasingGroupDescription_KR  Blocked_stock  \
0                      테이블 & 책상           3051   
1                            옷장          14629   
2                   TV 및 미디어 가구           2262   
3                           의자들           1200   
4                      테이블 & 책상          34657   

                                     Block_reason_KR Blocked_date  \
0  출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...   2024-06-06   
1   공급 업체의 지연으로 인해 재고 부족이 발생하여 주문 이행 능력에 영향을 미치는 경험.   2024-03-19   
2  추가 문서 작성이 필요한 

In [41]:
query2 = "What can be solution for the delivery for packing defects for furniture"
response2 = ask_llm_en(query=query2, retrieval_augmented_generation=True, k=4)
print(response2)

   BlockID  MaterialNumber            MaterialName  \
0        2          250249                VITTSJ√ñ   
1       10        69276256  M√ñCKELBY / HENRIKSDAL   
2        4        39213915                 KLIMPEN   
3        7        29203889                  PLATSA   

               MaterialDescription PurchasingGroupDescription  Blocked_stock  \
0          Laptop stand,  35x65 cm             Tables & desks          34657   
1  Table and 6 chairs,  235x100 cm             Tables & desks           3051   
2                Table,  150x75 cm             Tables & desks          21111   
3         Wardrobe,  140x57x261 cm                  Wardrobes          14629   

                                        Block_reason Blocked_date  \
0  Identification of packaging defects that compr...   2024-03-10   
1  Identification of packaging defects that compr...   2024-06-06   
2  Critical feedback on packaging quality leading...   2024-06-20   
3  Experiencing supplier delays that result in in...